In [10]:
import pandas as pd
import numpy as np
import sklearn as sk 
import opendatasets 
import matplotlib.pyplot as plt
import datetime


In [3]:
#Downloads the dataset to the current folder
import opendatasets 
ccfd_dataset_url = "https://www.kaggle.com/datasets/kartik2112/fraud-detection?select=fraudTrain.csv"
opendatasets.download(ccfd_dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading fraud-detection.zip to .\fraud-detection


100%|██████████| 202M/202M [05:45<00:00, 613kB/s] 


In [7]:
transactions_train = pd.read_csv("./fraud-detection/fraudTrain.csv")
transactions_test = pd.read_csv("./fraud-detection/fraudTest.csv")
transactions_train.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06
mean,6.483370e+05,4.171920e+17,7.035104e+01,4.880067e+04,3.853762e+01,-9.022634e+01,8.882444e+04,1.349244e+09,3.853734e+01,-9.022646e+01,5.788652e-03
std,3.743180e+05,1.308806e+18,1.603160e+02,2.689322e+04,5.075808e+00,1.375908e+01,3.019564e+05,1.284128e+07,5.109788e+00,1.377109e+01,7.586269e-02
min,0.000000e+00,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902779e+01,-1.666712e+02,0.000000e+00
25%,3.241685e+05,1.800429e+14,9.650000e+00,2.623700e+04,3.462050e+01,-9.679800e+01,7.430000e+02,1.338751e+09,3.473357e+01,-9.689728e+01,0.000000e+00
50%,6.483370e+05,3.521417e+15,4.752000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.456000e+03,1.349250e+09,3.936568e+01,-8.743839e+01,0.000000e+00
75%,9.725055e+05,4.642255e+15,8.314000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.359385e+09,4.195716e+01,-8.023680e+01,0.000000e+00
max,1.296674e+06,4.992346e+18,2.894890e+04,9.978300e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.371817e+09,6.751027e+01,-6.695090e+01,1.000000e+00


In [50]:
#Performs feature selection, engineering and extraction
def process_features(df: pd.DataFrame)->pd.DataFrame:
    new_df = df.drop(columns=["Unnamed: 0", "cc_num", "city", "street", "state", "zip", "trans_num"], axis=1)
    #Generate age and drop dob
    current_year = datetime.datetime.now().year
    new_df["age"]= new_df["dob"].apply(lambda x: current_year - int(x.split("-")[0]))
    new_df = new_df.drop(columns=["dob"], axis=1)
    #Remove the weird fraud_ thing that comes before a merchants name and rename merchant to merch_name
    new_df["merchant"] = new_df["merchant"].apply(lambda x: x.split("fraud_")[1]) 
    new_df = new_df.rename(columns={"merchant":"merch_name"})
    #Combine first and last name into full name then drop the them
    new_df["full_name"] = new_df["first"]+" "+new_df["last"] 
    new_df = new_df.drop(columns=["first", "last"], axis=1)
    #Drop unix time
    new_df = new_df.drop(columns=["unix_time"], axis=1)
    datetime_format = "%Y-%m-%d %H:%M:%S"
    new_df["dow"] = new_df["trans_date_trans_time"].apply(lambda x: datetime.datetime.strptime(x, datetime_format).weekday())
    new_df["dom"] = new_df["trans_date_trans_time"].apply(lambda x: int(x.split("-")[2].split(" ")[0]))
    new_df["month"] = new_df["trans_date_trans_time"].apply(lambda x: int(x.split("-")[1]))
    new_df["year"] = new_df["trans_date_trans_time"].apply(lambda x: int(x.split("-")[0]))
    def process_time(time:str)->float:
        times = time.split(" ")[1].split(":")
        return float(times[0])+float(times[1])/60
        #return float(times[0])+float(times[1])/60+float(times[2])/3600
    new_df["time"] = new_df["trans_date_trans_time"].apply(process_time)
    new_df.drop(columns=["trans_date_trans_time"], axis=1)
    new_column_order = ["full_name", "gender", "age", "job", "lat", "long", "city_pop", "merch_name", "merch_lat", "merch_long", "amt", "category", "time", "dow", "dom", "month", "year", "is_fraud"]
    return new_df.reindex(columns=new_column_order)
    


process_features(transactions_train)



,full_name,gender,age,job,lat,long,city_pop,merch_name,merch_lat,merch_long,amt,category,time,dow,dom,month,year,is_fraud
0,Jennifer Banks,F,35,"Psychologist, counselling",36.0788,-81.1781,3495,"Rippin, Kub and Mann",36.011293,-82.048315,4.97,misc_net,0.000000,1,1,1,2019,0
1,Stephanie Gill,F,45,Special educational needs teacher,48.8878,-118.2105,149,"Heller, Gutmann and Zieme",49.159047,-118.186462,107.23,grocery_pos,0.000000,1,1,1,2019,0
2,Edward Sanchez,M,61,Nature conservation officer,42.1808,-112.2620,4154,Lind-Buckridge,43.150704,-112.154481,220.11,entertainment,0.000000,1,1,1,2019,0
3,Jeremy White,M,56,Patent attorney,46.2306,-112.1138,1939,"Kutch, Hermiston and Farrell",47.034331,-112.561071,45.00,gas_transport,0.016667,1,1,1,2019,0
4,Tyler Garcia,M,37,Dance movement psychotherapist,38.4207,-79.4629,99,Keeling-Crist,38.674999,-78.632459,41.96,misc_pos,0.050000,1,1,1,2019,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,Erik Patterson,M,62,Geoscientist,37.7175,-112.4777,258,Reichel Inc,36.841266,-111.690765,15.56,entertainment,12.200000,6,21,6,2020,0
1296671,Jeffrey White,M,44,"Production assistant, television",39.2667,-77.5101,100,Abernathy and Sons,38.906881,-78.246528,51.70,food_dining,12.200000,6,21,6,2020,0
1296672,Christopher Castaneda,M,56,Naval architect,32.9396,-105.8189,899,Stiedemann Ltd,33.619513,-105.130529,105.93,food_dining,12.200000,6,21,6,2020,0
1296673,Joseph Murray,M,43,Volunteer coordinator,43.3526,-102.5411,1126,"Reinger, Weissnat and Strosin",42.788940,-103.241160,74.90,food_dining,12.216667,6,21,6,2020,0


In [42]:
transactions_processed_features = transactions_train.drop(columns=["Unnamed: 0", "cc_num", "city", "street", "state", "zip", "trans_num"], axis=1)
current_year = datetime.datetime.now().year
#Generate age and drop date of birth
transactions_processed_features["age"] =  transactions_processed_features["dob"].apply(lambda x: current_year - int(x.split("-")[0]))
transactions_processed_features = transactions_processed_features.drop(columns=["dob"], axis=1)
transactions_processed_features = transactions_processed_features.rename(columns={"merchant":"merch_name"})
transactions_processed_features["merch_name"] =  transactions_processed_features["merch_name"].apply(lambda x: x.split("fraud_")[1])
#Combine first name and last name for easier potential encoding
transactions_processed_features["full_name"] = transactions_processed_features["first"]+" "+transactions_processed_features["last"]
transactions_processed_features =transactions_processed_features.drop(columns=["first", "last"], axis=1)
transactions_processed_features = transactions_processed_features.drop(columns=["unix_time"], axis=1)

#Engineer the date_time of the transactions into a format more understandable tp a model
datetime_format = "%Y-%m-%d %H:%M:%S"
#This is kinda inefficient since it creates datetime object for 1.3 M transactions 4 times, I could have manually extracted what I need using string formatting techniques but I've chosen abstraction in favour of efficiency so that in the case that a dataset with a different datetime format is used all that is changed is the datetime_format string
transactions_processed_features["day_of_week"] = transactions_processed_features["trans_date_trans_time"].apply(lambda x: datetime.datetime.strptime(x, datetime_format).weekday())
transactions_processed_features["day_of_month"] = transactions_processed_features["trans_date_trans_time"].apply(lambda x: datetime.datetime.strptime(x, datetime_format).day)
transactions_processed_features["month"] = transactions_processed_features["trans_date_trans_time"].apply(lambda x: datetime.datetime.strptime(x, datetime_format).month)
transactions_processed_features["year"] = transactions_processed_features["trans_date_trans_time"].apply(lambda x: datetime.datetime.strptime(x, datetime_format).year)
transactions_processed_features



,trans_date_trans_time,merch_name,category,amt,gender,lat,long,city_pop,job,merch_lat,merch_long,is_fraud,age,full_name,day_of_week,day_of_month,month,year
0,2019-01-01 00:00:18,"Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,"Psychologist, counselling",36.011293,-82.048315,0,35,Jennifer Banks,1,1,1,2019
1,2019-01-01 00:00:44,"Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,Special educational needs teacher,49.159047,-118.186462,0,45,Stephanie Gill,1,1,1,2019
2,2019-01-01 00:00:51,Lind-Buckridge,entertainment,220.11,M,42.1808,-112.2620,4154,Nature conservation officer,43.150704,-112.154481,0,61,Edward Sanchez,1,1,1,2019
3,2019-01-01 00:01:16,"Kutch, Hermiston and Farrell",gas_transport,45.00,M,46.2306,-112.1138,1939,Patent attorney,47.034331,-112.561071,0,56,Jeremy White,1,1,1,2019
4,2019-01-01 00:03:06,Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,Dance movement psychotherapist,38.674999,-78.632459,0,37,Tyler Garcia,1,1,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,2020-06-21 12:12:08,Reichel Inc,entertainment,15.56,M,37.7175,-112.4777,258,Geoscientist,36.841266,-111.690765,0,62,Erik Patterson,6,21,6,2020
1296671,2020-06-21 12:12:19,Abernathy and Sons,food_dining,51.70,M,39.2667,-77.5101,100,"Production assistant, television",38.906881,-78.246528,0,44,Jeffrey White,6,21,6,2020
1296672,2020-06-21 12:12:32,Stiedemann Ltd,food_dining,105.93,M,32.9396,-105.8189,899,Naval architect,33.619513,-105.130529,0,56,Christopher Castaneda,6,21,6,2020
1296673,2020-06-21 12:13:36,"Reinger, Weissnat and Strosin",food_dining,74.90,M,43.3526,-102.5411,1126,Volunteer coordinator,42.788940,-103.241160,0,43,Joseph Murray,6,21,6,2020
